In [ ]:
from bs4 import BeautifulSoup
import requests
import re
from collections import Counter
import json
import networkx as nx
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time

G = nx.read_gpickle("semantic_graphs/skin_cancer.gpickle")
G2 = nx.read_gpickle("semantic_graphs/clinical_trials.gpickle")

In [ ]:
def non_internal_link_extraction(url, html):
    domain_regex = re.compile("(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\.)+[a-z0-9][a-z0-9-]{0,61}[a-z0-9]")
    soup = BeautifulSoup(html,"html.parser")
    non_internal_links = []
    for link_ in soup.find_all('a'):
        try:
            #Need to exclude internal links that do not contain the primary domain - only /xyz
            #Need to filter out advertising/partner/affiliate/links
            if re.search(domain_regex,link_.get('href')).group(0) != re.search(domain_regex,link).group(0):
                #print(re.search(domain_regex,link_.get('href')).group(0))
                non_internal_links.append(link_.get('href'))
        except Exception as e:
            pass
    
    return non_internal_links


def get_cancer_lexicon():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'}
    try:
        dict_response = requests.get("https://www.ncbi.nlm.nih.gov/books/NBK65951/#IX-Z", headers=headers)
        dict_txt = dict_response.content
        if (response.status_code == requests.codes.ok):
            pass
    except Exception as e:
        print(e)
        

    dict_soup = BeautifulSoup(dict_txt, "html.parser")

    for div in dict_soup.find_all("div"):
        try:
            if div.has_attr('class') and div['class'][0] == 'col six_col':
                print("Found First Column")
            elif div.has_attr('class') and div['class'][0] == 'col six_col last':
                print("Found First Column")
        except Exception as e:
            print(e)

    cancer_lexicon = Counter()

    for div in dict_soup.find_all("div"):
        if div.has_attr('class'):
            if "col" in div["class"] and "six_col" in div["class"] and len(div["class"]) == 2:
                for list_item in div.findAll('li'):
                    for hyperlink in list_item.findAll('a'):
                        cancer_lexicon[hyperlink.getText()] += 1

            elif "col" in div["class"] and "six_col" in div["class"] and "last" in div["class"] and len(div["class"]) == 3:
                for list_item in div.findAll('li'):
                    for hyperlink in list_item.findAll('a'):
                        cancer_lexicon[hyperlink.getText()] += 1
                        
    cancer_lexicon_list = list(cancer_lexicon.keys())
    return cancer_lexicon_list


cancer_lexicon_list = get_cancer_lexicon()
    
    

name 'response' is not defined
list index out of range


In [ ]:
import pandas as pd

In [ ]:
wiki_df = pd.read_csv("data/wikipedia-iso-country-codes.csv")

In [ ]:
import random
def random_walk(start_node, number_of_walks, DG):
    k = nx.dag_longest_path_length(DG)
    print(k)
    visited_nodes = Counter()
    visited_edges = Counter()
    #prev_node = start_node
    n_steps = 0
    #To Mitigate the effects of randomness
    for j in range(1000):
        prev_node = start_node
        for i in range(number_of_walks):
            prev_node = start_node
            n_steps = 0
            while True:
                try:
                    #print(list(DG.out_edges(prev_node,data=True)))
                    edge_traversed = random.choice(list(DG.out_edges(prev_node,data=True)))
                    #print(edge_traversed)
                    node = edge_traversed[1]
                    visited_nodes[node] += 1
                    visited_edges[(edge_traversed[2]['relationship'],node)] += 1
                except IndexError as e:
                    #print(e)
                    pass
                n_steps += 1
                if n_steps >= k:
                    break
                else:
                    try: 
                        prev_node = node
                    except UnboundLocalError as e:
                        pass

    visited_nodes = {k: v / 1000 for k,v in visited_nodes.items()}
    visited_edges = {k: v / 1000 for k,v in visited_edges.items()}
    
    return visited_nodes,visited_edges

In [ ]:
{**{"a": 1}, **{"b" : 2}}

{'a': 1, 'b': 2}

In [ ]:
import pandas as pd
medical_journals = pd.read_csv("data/scimagojr 2020 Subject Category - Medicine (miscellaneous).csv", sep = ";")

medical_journals[medical_journals["Title"] == "Nature Medicine"]

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories
3,4,15819,Nature Medicine,journal,"1546170X, 10788956","19,536",Q1,547,452,953,10601,22548,664,"23,52","23,45",United Kingdom,Western Europe,Nature Publishing Group,1995-2020,"Biochemistry, Genetics and Molecular Biology (..."


In [52]:
import website_parser
import json
import textual_feature_extracter
import random

In [4]:
link = ["https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves", "https://www.independent.ie/life/health-wellbeing/living-with-cancer/if-i-hadnt-lost-four-and-a-half-stone-i-would-never-have-found-the-lump-and-caught-my-breast-cancer-40907808.html"
,"https://www.irishtimes.com/news/health/more-late-stage-cancers-being-diagnosed-after-screening-delays-charity-says-1.4713292"
,"https://medicalfuturist.com/technologies-that-will-shape-the-future-of-cancer-care/"
,"https://www.rte.ie/news/health/2021/1111/1259179-cancer/"
,"https://www.irishnews.com/magazine/science/2021/11/15/news/childhood-cancer-survivors-at-greater-risk-of-ill-health-as-they-age-study-2507553/"
,"https://www.news-medical.net/news/20211115/Exercise-reduces-arm-and-shoulder-disability-pain-in-women-recovering-after-breast-cancer-surgery.aspx"
,"https://med.stanford.edu/cancer/trials/vaccine.html"
,"https://www.health.harvard.edu/blog/a-new-targeted-treatment-for-early-stage-breast-cancer-202111242647"
,"https://www.everydayhealth.com/cancer/cancer-news-digest-latest-developments-in-cancer-research-and-treatment-for-october/"
,"https://www.dermatologytimes.com/view/advances-in-skin-cancer-diagnosis-and-education"
,"https://www.reuters.com/business/healthcare-pharmaceuticals/biontechs-skin-cancer-therapy-wins-fdas-fast-track-designation-2021-11-19/"
,"https://newatlas.com/medical/princeton-team-long-targeted-gene-metastasis-major-cancers/"
,"https://www.mayoclinic.org/diseases-conditions/skin-cancer/symptoms-causes/syc-20377605"
,"https://www.thebrighterside.news/post/game-changing-operation-can-cure-prostate-cancer"]


In [102]:
link = ["https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves", "https://www.independent.ie/life/health-wellbeing/living-with-cancer/if-i-hadnt-lost-four-and-a-half-stone-i-would-never-have-found-the-lump-and-caught-my-breast-cancer-40907808.html"
,"https://www.irishtimes.com/news/health/more-late-stage-cancers-being-diagnosed-after-screening-delays-charity-says-1.4713292"
,"https://medicalfuturist.com/technologies-that-will-shape-the-future-of-cancer-care/"
,"https://www.rte.ie/news/health/2021/1111/1259179-cancer/"
,"https://www.irishnews.com/magazine/science/2021/11/15/news/childhood-cancer-survivors-at-greater-risk-of-ill-health-as-they-age-study-2507553/"
,"https://www.news-medical.net/news/20211115/Exercise-reduces-arm-and-shoulder-disability-pain-in-women-recovering-after-breast-cancer-surgery.aspx"
,"https://med.stanford.edu/cancer/trials/vaccine.html"
,"https://www.health.harvard.edu/blog/a-new-targeted-treatment-for-early-stage-breast-cancer-202111242647"
,"https://www.everydayhealth.com/cancer/cancer-news-digest-latest-developments-in-cancer-research-and-treatment-for-october/"
,"https://www.dermatologytimes.com/view/advances-in-skin-cancer-diagnosis-and-education"
,"https://www.reuters.com/business/healthcare-pharmaceuticals/biontechs-skin-cancer-therapy-wins-fdas-fast-track-designation-2021-11-19/"
,"https://newatlas.com/medical/princeton-team-long-targeted-gene-metastasis-major-cancers/"
,"https://www.mayoclinic.org/diseases-conditions/skin-cancer/symptoms-causes/syc-20377605"
,"https://www.thebrighterside.news/post/game-changing-operation-can-cure-prostate-cancer"]

choice = random.choice(link[0:2])
my_parser = website_parser.WebsiteParser(choice)
payload = my_parser.get_payload()
my_text_extractor = textual_feature_extracter.TextualFeatureExtractor(payload)
sentence_map = my_text_extractor.process_sentence_map()
print(choice)
sentence_map


https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves


{'0.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '1.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '2.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '3.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '4.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '5.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '6.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '7.0': {'keywords': 'no_keywords_found', 'entities': {'GNI': (0, 3, 'ORG')}},
 '8.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '9.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '10.0': {'keywords': 'no_keywords_found',
  'entities': {'The Brighter Side of News': (0, 25, 'PERSON')}},
 '11.0': {'keywords': 'no_keywords_found',
  'entities': {'University of Zurich': (16, 36, 'ORG')}},
 '12.0': {'keywords': {'refe

In [106]:
payload

'{"sentence_map": {"0.0": "HOME", "1.0": "GOOD NEWS", "2.0": "DISCOVERIES", "3.0": "INNOVATIONS", "4.0": "GLOBAL GOOD", "5.0": "GREEN IMPACT", "6.0": "CELEBRITIES", "7.0": "GNI", "8.0": "PODCAST", "9.0": "More", "10.0": "The Brighter Side of News features good news, compassionate news\\u00a0 and inspirational news from around the world", "11.0": "[Nov 13, 2021:  University of Zurich]", "12.0": "A new technology developed by UZH researchers enables the body to produce therapeutic agents on demand at the exact location where they are needed.", "12.1": "The innovation could reduce the side effects of cancer therapy and may hold the solution to better delivery of Covid-related therapies directly to the lungs.", "13.0": "Scientists at the University of Zurich have modified a common respiratory virus, called adenovirus, to act like a Trojan horse to deliver genes for cancer therapeutics directly into tumor cells.", "13.1": "Unlike chemotherapy or radiotherapy, this approach does no harm to n

In [ ]:
from itertools import combinations

In [ ]:
def pairwise_path_comparison(p1,p2):
    step_comparison = []
    counter = 0

    for s1, s2 in zip(p1,p2):
        if counter % 2 == 1:
            if s1 == s2:
                step_comparison.append(1)
            else:
                step_comparison.append(0)
        counter += 1    

    return step_comparison
            

    
    


In [ ]:
import spacy
from spacy import displacy
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")


In [1]:
import textual_feature_extracter
import random
import website_parser
choice = "https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves"
my_parser = website_parser.WebsiteParser(choice)
payload = my_parser.get_payload()
my_text_extractor = textual_feature_extracter.TextualFeatureExtractor(payload)
sentence_map = my_text_extractor.process_sentence_map()



{'0.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '1.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '2.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '3.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '4.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '5.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '6.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '7.0': {'keywords': 'no_keywords_found', 'entities': {'GNI': (0, 3, 'ORG')}},
 '8.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '9.0': {'keywords': 'no_keywords_found', 'entities': 'no_entities_found'},
 '10.0': {'keywords': 'no_keywords_found',
  'entities': {'The Brighter Side of News': (0, 25, 'PERSON')}},
 '11.0': {'keywords': 'no_keywords_found',
  'entities': {'University of Zurich': (16, 36, 'ORG')}},
 '12.0': {'keywords': {'refe

In [1]:
import website_parser


website_parser.WebsiteParser("https://www.irishtimes.com/news/health/more-late-stage-cancers-being-diagnosed-after-screening-delays-charity-says-").get_payload()

'{"error": "404 Page not found - That is not a valid url"}'

In [11]:
import textual_feature_transformer
import pprint
pp = pprint.PrettyPrinter(indent=4)
my_tft = textual_feature_transformer.TextualFeatureTransformer()
import json
sentences_overlooked = []



for choice in link:
    my_parser = website_parser.WebsiteParser(choice)
    payload = my_parser.get_payload()
    my_text_extractor = textual_feature_extracter.TextualFeatureExtractor(payload)
    sentence_map = my_text_extractor.process_sentence_map()
    json_payload = json.loads(payload)["sentence_map"]
    first_loop = True
    past_index = None

    for par_sent_key, features in sentence_map.items():
        current_index = [int(number) for number in par_sent_key.split(".")]
        print(current_index, past_index)
        if first_loop:
            pass
        else:
            if current_index[0] != past_index[0]:
                print("New Paragraph")

            else:
                print("Same Paragraph")
            
            

        if features["keywords"] != "no_keywords_found":
            for k,v in features["keywords"].items():
                #print(k_,v_)
                if k in ["cancer_semantic_nodes","drug_development_semantic_nodes"]:
                    for keyword, count in v.items():
                        #print(keyword, count)
                        paths = my_tft.random_walk(keyword, count, k)
                        if paths == "keyword_is_terminal_node":
                            #add node directly to counter
                            #print(paths)
                            pass
                        else:
                            #need to do something with the tuples and their probability values
                            #print(paths)
                            pass

                    print()
                else:
                    
                    print(v)
                    pass
            print("-------------------------------------------------------------------------")
        else:
            sentences_overlooked.append(json_payload[par_sent_key])
        first_loop = False
        past_index = current_index

[0, 0] None
[1, 0] [0, 0]
New Paragraph
[2, 0] [1, 0]
New Paragraph
[3, 0] [2, 0]
New Paragraph
[4, 0] [3, 0]
New Paragraph
[5, 0] [4, 0]
New Paragraph
[6, 0] [5, 0]
New Paragraph
[7, 0] [6, 0]
New Paragraph
[8, 0] [7, 0]
New Paragraph
[9, 0] [8, 0]
New Paragraph
[10, 0] [9, 0]
New Paragraph
[11, 0] [10, 0]
New Paragraph
[12, 0] [11, 0]
New Paragraph
{'therapeutic': 1}
{('Universität Zürich', 'Université de Zurich', 'University of Zurich', 'UZH'): 1}
-------------------------------------------------------------------------
[12, 1] [12, 0]
Same Paragraph
{'cancer': 1, 'therapy': 1}

-------------------------------------------------------------------------
[13, 0] [12, 1]
New Paragraph
{'virus': 1, 'adenovirus': 1, 'cancer': 1, 'tumor': 1}

-------------------------------------------------------------------------
[13, 1] [13, 0]
Same Paragraph
{'chemotherapy': 1, 'radiotherapy': 1}

-------------------------------------------------------------------------
[13, 2] [13, 1]
Same Paragraph
{

In [12]:
for sent in sentences_overlooked:
    print(sent)

HOME
GOOD NEWS
DISCOVERIES
INNOVATIONS
GLOBAL GOOD
GREEN IMPACT
CELEBRITIES
GNI
PODCAST
More
The Brighter Side of News features good news, compassionate news  and inspirational news from around the world
[Nov 13, 2021:  University of Zurich]
Related Stories:
Use to combat Covid-19 being investigated
Adenoviral vectors are already being used in several of the COVID vaccines, including the Johnson & Johnson, AstraZeneca, China's CanSino Biologics and Russia's Sputnik V vaccines - but without the innovative SHREAD technology.
Like these kind of feel good stories?
Get the Brighter Side of News' newsletter.
Tags:
Most Recent Stories
© 2021 The Brighter Side of News
Site Map | Newsletter Sign-up | News Feed | About Us
 Independentie
 Search  Search
Wednesday, 12 January 2022 | 9.4°C Dublin
 Search  Search
  Menu Sections
 Search  Search
  Menu Sections
Marguerite Sloane found a lump just before she turned 50.
 Close
Marguerite Sloane
Photo: Paul McCarthy.
/
Arlene Harris
     October 04 2021

In [ ]:
def dependancy_parse_sweep(sentence, feature_map):
    for token in doc:
        try:
            for entity in feature_map["entities"].keys():
                if token.text == entity:
                    # We have an attribute and direct object, so check for subject
                    if token.dep_ in ("attr", "dobj"):
                        subj = [w for w in token.head.lefts if w.dep_ == "nsubj"]
                        if subj:
                            print(subj[0], "-->", token)
                    # We have a prepositional object with a preposition
                    elif token.dep_ == "pobj" and token.head.dep_ == "prep":
                        print(token.head.head, "-->", token)
        except:
            pass
        try:
            for entity in feature_map["keywords"]["cancer_semantic_nodes"].keys():
                if token.text == entity:
                    # We have an attribute and direct object, so check for subject
                    if token.dep_ in ("attr", "dobj"):
                        subj = [w for w in token.head.lefts if w.dep_ == "nsubj"]
                        if subj:
                            print(subj[0], "-->", token)
                    # We have a prepositional object with a preposition
                    elif token.dep_ == "pobj" and token.head.dep_ == "prep":
                        print(token.head.head, "-->", token)
        except:
            pass
                


In [30]:
my_tft.random_walk("cancer", 1, "cancer_semantic_nodes")

'keyword_is_terminal_node'

In [ ]:
list(paths_2.items())[1][0]

('immunotherapy',
 'is_a_type_of',
 'medication',
 'is_a_treatment_for',
 'merkel cell cancer',
 'is_a_type_of',
 'skin cancer',
 'is_a_type_of',
 'cancer')

In [50]:
def collapse_path(path):
    new_path = []
    edge_semantics = []
    edge_counter = 0
    semantic_edge = True
    path = list(path)

    for i in range(len(path)):
        if i % 2 == 0:
            candidate_nodes = []
            for j in range(i, len(path)):
                if j % 2 == 1:
                    print(j)
                    if path[j] in ["is_a_type_of", 'is_a_synonym_of']:
                        candidate_nodes.append(path[j+1])
                    else:
                        print(candidate_nodes)
                        break
                    

print(collapse_path(('immunotherapy', 'is_a_type_of', 'medication', 'is_a_treatment_for', 'basal cell carcinoma', 'is_a_type_of', 'keratinocyte carcinoma', 'is_a_type_of', 'skin cancer', 'is_a_type_of', 'cancer')))
print(collapse_path(('mice', 'are_used_for', 'in vivo', 'is_a_type_of_experiment_in', 'preclinical', 'participants_are', 'not human')))
print(collapse_path(('clinical trials', 'is_a_synonym_of', 'clinical trial', 'participants_are', 'human')))

1
3
['medication']
3
[]
5
7
9
7
9
9
None
1
[]
3
[]
5
[]
None
1
3
['clinical trial']
3
[]
None


In [37]:
all([True, True, True])

True

In [1]:
import tf_idf

In [2]:
my_tfidf = tf_idf.tf_idf()

NO TRAINED VECTORIZER EXISTS - CREATING NOW
Estimated Remaining Runtime 917.6104783999999 seconds
Estimated Remaining Runtime 686.973108 seconds
Estimated Remaining Runtime 590.190545 seconds
Estimated Remaining Runtime 556.314419325 seconds
Estimated Remaining Runtime 524.64864064 seconds
Estimated Remaining Runtime 505.91095445 seconds
Estimated Remaining Runtime 485.2424792857143 seconds
Estimated Remaining Runtime 467.472448525 seconds
Estimated Remaining Runtime 449.74771840000005 seconds
Estimated Remaining Runtime 437.78131915 seconds
Estimated Remaining Runtime 421.47477580000003 seconds
Estimated Remaining Runtime 408.758810875 seconds
Estimated Remaining Runtime 402.84034615384616 seconds
Estimated Remaining Runtime 393.84550266428573 seconds
Estimated Remaining Runtime 385.31020368 seconds
Estimated Remaining Runtime 378.03209068125 seconds
Estimated Remaining Runtime 371.27967882352937 seconds
Estimated Remaining Runtime 366.79631055 seconds
Estimated Remaining Runtime 361.

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
knn_abstract = RandomForestClassifier()
knn_title = RandomForestClassifier()
knn_abstract.fit(my_tfidf.X_abstract, my_tfidf.labels)
knn_title.fit(my_tfidf.X_titles, my_tfidf.labels)

RandomForestClassifier()

In [21]:
knn_abstract.score(my_tfidf.X_abstract, my_tfidf.labels)

0.9876346635950946

In [27]:
classifier_agreement = []
for l in link:
    my_parser = website_parser.WebsiteParser(l)
    payload = my_parser.get_payload()
    my_text_extractor = textual_feature_extracter.TextualFeatureExtractor(payload)
    sentence_map = my_text_extractor.process_sentence_map()
    print(my_tfidf.pre_processing(json.loads(payload)["article_slug"]))
    if knn_abstract.predict(my_tfidf.abstracts_vectorizer.transform([my_tfidf.pre_processing(json.loads(payload)["bag_of_words"])])) == knn_title.predict(my_tfidf.titles_vectorizer.transform([my_tfidf.pre_processing(json.loads(payload)["article_slug"])])):
        classifier_agreement.append(1)
        print(knn_abstract.predict(my_tfidf.abstracts_vectorizer.transform([my_tfidf.pre_processing(json.loads(payload)["bag_of_words"])])))
    else:
        classifier_agreement.append(0)

print(sum(classifier_agreement) / len(classifier_agreement))
    

post new technolog trick cancer tumor elimin
['skin+cancer']
life health wellb live cancer hadnt lost four half stone would never found lump caught breast cancer forti million nine hundr seven thousand eight hundr eight
['breast+cancer']
news health late stage cancer diagnos screen delay chariti say one
['skin+cancer']
technolog shape futur cancer care
['skin+cancer']
news health two thousand twenty-on one thousand one hundr eleven one million two hundr fifty-nin thousand one hundr seventy-nin cancer
magazin scienc two thousand twenty-on eleven fifteen news childhood cancer survivor greater risk ill health age studi two million five hundr seven thousand five hundr fifty-thre
['skin+cancer']
news twenti million two hundr eleven thousand one hundr fifteen exercis reduc arm shoulder disabl pain woman recov breast cancer surgeri
['breast+cancer']
cancer trial vaccin
['skin+cancer']
blog new target treatment earli stage breast cancer two hundr two billion one hundr eleven million two hundr 

In [19]:
import numpy as np


    


https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves
https://www.thebrighterside.news/post/abcdefhijklmno-pqrstuvwxyz
https://www.independent.ie/life/health-wellbeing/living-with-cancer/if-i-hadnt-lost-four-and-a-half-stone-i-would-never-have-found-the-lump-and-caught-my-breast-cancer-40907808.html
https://www.independent.ie/life/health-wellbeing/living-with-cancer/abcdefhijklmno-pqrstuvwxyz
https://www.irishtimes.com/news/health/more-late-stage-cancers-being-diagnosed-after-screening-delays-charity-says-1.4713292
https://www.irishtimes.com/news/health/abcdefhijklmno-pqrstuvwxyz
https://medicalfuturist.com/technologies-that-will-shape-the-future-of-cancer-care/
https://medicalfuturist.com/abcdefhijklmno-pqrstuvwxyz/
https://www.rte.ie/news/health/2021/1111/1259179-cancer/
https://www.rte.ie/news/health/2021/1111/abcdefhijklmno-pqrstuvwxyz/
https://www.irishnews.com/magazine/science/2021/11/15/news/childhood-cancer-survivors-at-greater

In [1]:
import website_parser

C:\Users\Colmj\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
for choice in link:
    web_parse = website_parser.WebsiteParser(choice[:-7])
    print(choice[:-7])
    print(web_parse.get_payload())
    print()
    

https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-the
{"error": "Soft 404 Page not found - That is not a valid url slug"}

https://www.independent.ie/life/health-wellbeing/living-with-cancer/if-i-hadnt-lost-four-and-a-half-stone-i-would-never-have-found-the-lump-and-caught-my-breast-cancer-409078
{"error": "404 Page not found - That is not a valid url"}

https://www.irishtimes.com/news/health/more-late-stage-cancers-being-diagnosed-after-screening-delays-charity-says-1.
{"error": "404 Page not found - That is not a valid url"}

https://medicalfuturist.com/technologies-that-will-shape-the-future-of-cance
{"error": "404 Page not found - That is not a valid url"}

https://www.rte.ie/news/health/2021/1111/1259179-
{"sentence_map": {"0.0": "Ireland's National Public Service Media", "1.0": "Me\u00e1in N\u00e1isi\u00fanta Seirbh\u00edse Poibl\u00ed na h\u00c9ireann", "2.0": "Researchers at Queen's University in Belfast have developed a new treatment

In [3]:
link = ["https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves", "https://www.independent.ie/life/health-wellbeing/living-with-cancer/if-i-hadnt-lost-four-and-a-half-stone-i-would-never-have-found-the-lump-and-caught-my-breast-cancer-40907808.html"
,"https://www.irishtimes.com/news/health/more-late-stage-cancers-being-diagnosed-after-screening-delays-charity-says-1.4713292"
,"https://medicalfuturist.com/technologies-that-will-shape-the-future-of-cancer-care/"
,"https://www.rte.ie/news/health/2021/1111/1259179-cancer/"
,"https://www.irishnews.com/magazine/science/2021/11/15/news/childhood-cancer-survivors-at-greater-risk-of-ill-health-as-they-age-study-2507553/"
,"https://www.news-medical.net/news/20211115/Exercise-reduces-arm-and-shoulder-disability-pain-in-women-recovering-after-breast-cancer-surgery.aspx"
,"https://med.stanford.edu/cancer/trials/vaccine.html"
,"https://www.health.harvard.edu/blog/a-new-targeted-treatment-for-early-stage-breast-cancer-202111242647"
,"https://www.everydayhealth.com/cancer/cancer-news-digest-latest-developments-in-cancer-research-and-treatment-for-october/"
,"https://www.dermatologytimes.com/view/advances-in-skin-cancer-diagnosis-and-education"
,"https://www.reuters.com/business/healthcare-pharmaceuticals/biontechs-skin-cancer-therapy-wins-fdas-fast-track-designation-2021-11-19/"
,"https://newatlas.com/medical/princeton-team-long-targeted-gene-metastasis-major-cancers/"
,"https://www.mayoclinic.org/diseases-conditions/skin-cancer/symptoms-causes/syc-20377605"
,"https://www.thebrighterside.news/post/game-changing-operation-can-cure-prostate-cancer"]

In [7]:
import website_parser
import json


for choice in link:
    my_parser = website_parser.WebsiteParser(link[-1])
    payload = my_parser.get_payload()
    print(json.loads(payload))
    print("---")
    break

{'sentence_map': {'0.0': 'HOME', '1.0': 'GOOD NEWS', '2.0': 'DISCOVERIES', '3.0': 'INNOVATIONS', '4.0': 'GLOBAL GOOD', '5.0': 'GREEN IMPACT', '6.0': 'CELEBRITIES', '7.0': 'GNI', '8.0': 'PODCAST', '9.0': 'More', '10.0': 'The Brighter Side of News features good news, compassionate news\xa0 and inspirational news from around the world', '11.0': '[Jan 3, 2021: Emily Brown]', '12.0': 'Doctors in the UK hope that thousands of men with prostate cancer could be cured through an innovative hour-long operation.', '13.0': "The 'game-changing' treatment uses electrical currents to destroy difficult to reach tumours.", '13.1': 'The one-hour ‘Nanoknife’ operation has been described as ‘amazingly simple and quick’ by surgeons.', '14.0': 'It uses a technique called irreversible electroporation to administer electrical pulses into the tumour, cutting open the membrane of the cells in a far less invasive manner than standard treatments, meaning there are fewer risks to surrounding organs and tissues.', 

In [2]:
import website_parser

C:\Users\Colmj\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
my_parser = website_parser.WebsiteParser(link[0])

In [4]:
my_parser.get_payload()

{'property': 'og:title', 'content': 'New technology tricks cancerous tumors into eliminating themselves'}
{'property': 'og:description', 'content': 'Researchers have modified a common respiratory virus, called adenovirus, to act like a Trojan horse to deliver genes for cancer therapeutics'}
{'property': 'og:image', 'content': 'https://static.wixstatic.com/media/95ac29_63f57401d4ea4719bf7794906cd7f7b7~mv2.jpg/v1/fit/w_1000%2Ch_800%2Cal_c%2Cq_80/file.jpg'}
{'property': 'og:image:width', 'content': '1200'}
{'property': 'og:image:height', 'content': '800'}
{'property': 'og:url', 'content': 'https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves'}
{'property': 'og:site_name', 'content': 'Brighter Side News'}
{'property': 'og:type', 'content': 'article'}
{'property': 'article:author', 'content': 'Joseph Shavit'}
{'property': 'article:published_time', 'content': '2021-11-13T15:52:36.683Z'}
{'property': 'article:modified_time', 'content': '2021

'{"sentence_map": {"0.0": "HOME", "1.0": "GOOD NEWS", "2.0": "DISCOVERIES", "3.0": "INNOVATIONS", "4.0": "GLOBAL GOOD", "5.0": "GREEN IMPACT", "6.0": "CELEBRITIES", "7.0": "GNI", "8.0": "PODCAST", "9.0": "More", "10.0": "The Brighter Side of News features good news, compassionate news\\u00a0 and inspirational news from around the world", "11.0": "[Nov 13, 2021:  University of Zurich]", "12.0": "A new technology developed by UZH researchers enables the body to produce therapeutic agents on demand at the exact location where they are needed.", "12.1": "The innovation could reduce the side effects of cancer therapy and may hold the solution to better delivery of Covid-related therapies directly to the lungs.", "13.0": "Scientists at the University of Zurich have modified a common respiratory virus, called adenovirus, to act like a Trojan horse to deliver genes for cancer therapeutics directly into tumor cells.", "13.1": "Unlike chemotherapy or radiotherapy, this approach does no harm to n

In [5]:
my_parser.get_all_metadata()

{'property': 'og:title', 'content': 'New technology tricks cancerous tumors into eliminating themselves'}
{'property': 'og:description', 'content': 'Researchers have modified a common respiratory virus, called adenovirus, to act like a Trojan horse to deliver genes for cancer therapeutics'}
{'property': 'og:image', 'content': 'https://static.wixstatic.com/media/95ac29_63f57401d4ea4719bf7794906cd7f7b7~mv2.jpg/v1/fit/w_1000%2Ch_800%2Cal_c%2Cq_80/file.jpg'}
{'property': 'og:image:width', 'content': '1200'}
{'property': 'og:image:height', 'content': '800'}
{'property': 'og:url', 'content': 'https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves'}
{'property': 'og:site_name', 'content': 'Brighter Side News'}
{'property': 'og:type', 'content': 'article'}
{'property': 'article:author', 'content': 'Joseph Shavit'}
{'property': 'article:published_time', 'content': '2021-11-13T15:52:36.683Z'}
{'property': 'article:modified_time', 'content': '2021

{'title': 'New technology tricks cancerous tumors into eliminating themselves',
 'description': 'Researchers have modified a common respiratory virus, called adenovirus, to act like a Trojan horse to deliver genes for cancer therapeutics',
 'url': 'https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves',
 'site_name': 'Brighter Side News',
 'content_type': 'article',
 'author': 'Joseph Shavit',
 'published_time': '2021-11-13T15:52:36.683Z',
 'modified_time': '2021-11-13T15:52:36.683Z',
 'language_territory': None,
 'tags': None,
 'experiation_time': None,
 'section': None}

In [7]:
a = set([1,2,3])
b=  set([3,4,5])

In [8]:
a.difference(b)

{1, 2}

In [7]:
import website_parser
import textual_feature_extracter



'https://www.thebrighterside.news/post/new-technology-tricks-cancerous-tumors-into-eliminating-themselves'

In [33]:
my_parser = website_parser.WebsiteParser("https://www.express.co.uk/life-style/health/1555460/Cancer-symptoms-antibiotics-prostate-breast-lung-colon-risk")
payload = my_parser.get_payload()

In [36]:
my_parser.get_all_sentences()

{'2.0': 'Antibiotics are one of the most widely used types of medicines.',
 '3.0': 'There are large differences in outpatient antibiotic use across countries.',
 '4.0': 'It has been suggested that up to 75 percent of antibiotic use has a questionable therapeutic value.',
 '7.0': 'Antibiotics have an effect on the composition and functioning of intestinal microbiota and on inflammation and immune functions',
 '8.0': 'Through these effects antibiotics could play a role in the development of many types of cancer.',
 '9.0': 'Given the widespread use of antibiotics and high incidence of cancer—over 10 million new cancer cases occur each year worldwide—it is important to determine whether there is an association between the use of antibiotics and the risk of cancer.',
 '11.0': 'In a study published in Wiley Online, antibiotic use predicting an increased risk of cancer was analysed.',
 '12.0': 'The nationwide cohort study included 3,112,624 individuals, aged 30–79 years, with no history of ca

In [19]:
my_text_extractor = textual_feature_extracter.TextualFeatureExtractor(payload)
my_text_extractor.process_sentence_map()

{'1.0': {'keywords': {'referenced_cancer_keywords': {'virus': 1,
    'adenovirus': 1,
    'cancer': 1,
    'tumor': 1},
   'cancer_semantic_nodes': {'cancer': 1}},
  'entities': {'the University of Zurich': (14, 38, 'ORG'),
   'Trojan': (114, 120, 'PRODUCT')}},
 '1.1': {'keywords': {'referenced_cancer_keywords': {'chemotherapy': 1,
    'radiotherapy': 1},
   'cancer_semantic_nodes': {'chemotherapy': 1}},
  'entities': 'no_entities_found'},
 '2.0': {'keywords': {'referenced_cancer_keywords': {'tumor': 1,
    'therapeutic': 1,
    'cancer': 1},
   'cancer_semantic_nodes': {'cancer': 1}},
  'entities': 'no_entities_found'},
 '3.0': {'keywords': {'referenced_cancer_keywords': {'tumor': 1}},
  'entities': {'Sheena Smith': (133, 145, 'PERSON')}},
 '4.0': {'keywords': {'referenced_cancer_keywords': {'therapeutic': 2}},
  'entities': {'Andreas Plückthun': (22, 39, 'PERSON')}},
 '5.0': {'keywords': {'referenced_universities': {('Universität Zürich',
     'Université de Zurich',
     'University

In [30]:
from rich import print
import json
